In [1]:
from jax import random, numpy as jnp
from stadion.models import LinearSDE, MLPSDE
from pprint import pprint
from stadion import kds_loss
import numpy as onp
import ot

2025-02-13 19:20:57.345575: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1739492457.356613 3498570 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1739492457.359894 3498570 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
from nn import NNSDE
from nonlinear_script import *

In [3]:
n = 500
d = 20
r = 2
n_envs = 10
n_test_envs = 10
epsilon = 0.5
gamma = 0.98
activation = "mixed"
model_activation = "learned"

model_hidden_size = 8

bandwidth = 7.0
steps = 30000
estimator = "linear"
reg = 0.00

intv_scale = 0.3

n_samples_burnin=200
learning_rate = 0.003
weight_decay = 0.0001

seed = 4

In [4]:
key = random.PRNGKey(seed)

true_model = build_model(r, activation, epsilon, gamma, n_samples_burnin)
key, true_model, param = initialize_model(key, true_model, d)

# param["mlp_0"] = jax.numpy.eye(d, r)
# param["mlp_1"] = jax.numpy.eye(r, d)
# true_model.param = param

key, intv_param, datasets, targets = build_data(key, true_model, d, n_envs, intv_scale, n)
key, test_intv_param, test_datasets, test_targets = build_data(key, true_model, d, n_test_envs, intv_scale, n)

model = build_model(model_hidden_size, model_activation, epsilon, gamma, n_samples_burnin)
model = fit_model(model, datasets, targets, intv_param, bandwidth, steps, estimator, reg, learning_rate, weight_decay)

W0000 00:00:1739492480.777438 3498570 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


step:  3000 kds:     0.144117  | min remain:  2.4  sec/step: 0.005
step:  6000 kds:     0.003861  | min remain:  1.7  sec/step: 0.004
step:  9000 kds:     0.003837  | min remain:  1.3  sec/step: 0.004
step: 12000 kds:     0.002486  | min remain:  1.1  sec/step: 0.004
step: 15000 kds:     0.002251  | min remain:  0.9  sec/step: 0.004
step: 18000 kds:     0.001534  | min remain:  0.8  sec/step: 0.004
step: 21000 kds:     0.001232  | min remain:  0.6  sec/step: 0.004
step: 24000 kds:     0.000688  | min remain:  0.4  sec/step: 0.004
step: 27000 kds:     0.000523  | min remain:  0.2  sec/step: 0.004
step: 30000 kds:     0.000323  | min remain:  0.0  sec/step: 0.004


In [5]:
wds, mean, std = assess_model_mean(key, model, n_test_envs, test_intv_param, test_datasets, n)
print(wds)
print(mean, std)

[14.749057  26.890985  21.422445   7.1079946 19.149784  26.557318
 13.632009  14.202805   6.4545097  9.287248 ]
15.945416 7.03571


In [6]:
wds, mean, std = assess_model(key, model, n_test_envs, test_intv_param, test_datasets, n, ot_epsilon = 0.2)
print(wds)
print(mean, std)

[222.19447  728.02966  464.54572   59.21233  388.3087   711.5851
 190.4717   206.38905   52.976326  94.95804 ]
311.8671 239.56877


In [7]:
wds, mean, std = assess_model_mean(key, true_model, n_test_envs, test_intv_param, test_datasets, n)
print(wds)
print(mean, std)

[1.4234011  0.09190805 0.7598879  0.41335633 0.7564064  1.5509962
 1.5348772  0.88837326 0.19383423 0.11704987]
0.773009 0.5460264


In [8]:
# wds, mean, std = assess_model(key, true_model, n_test_envs, test_intv_param, test_datasets, n, ot_epsilon = 0.2)
# print(wds)
# print(mean, std)

In [9]:
#s:1.4552428 0.851895
#l:
#t:0.8322261 0.67133

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

if d == 2:

    total = 3

    pred_datasets = []
    for k in range(total):
        local_intv_param = intv_param.index_at(k)
        key, subk = random.split(key)
        data = model.sample(subk, n_samples = n, intv_param = local_intv_param)
        pred_datasets.append(data)

    fig, axes = plt.subplots(nrows=total,ncols=2,sharex=True, sharey=True)

    
    for k in range(total):
    
        x = datasets[k][:,0]
        y = datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,0], fill=True,cmap="Purples_r")
        axes[k,0].set_title(f"True model on training env {k}")
        
        x = pred_datasets[k][:,0]
        y = pred_datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,1], fill=True,cmap="Purples_r")
        axes[k,1].set_title(f"Trained model on training env {k}")

    plt.tight_layout()
    plt.show()


In [11]:
if d == 2:

    total = 3

    pred_test_datasets = []
    for k in range(total):
        local_intv_param = test_intv_param.index_at(k)
        key, subk = random.split(key)
        data = model.sample(subk, n_samples = n, intv_param = local_intv_param)
        pred_test_datasets.append(data)

    fig, axes = plt.subplots(nrows=total,ncols=2,sharex=True, sharey=True)

    for k in range(total):
    
        x = test_datasets[k][:,0]
        y = test_datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,0], fill=True,cmap="Purples_r")
        axes[k,0].set_title(f"True model on test env {k}")
        
        x = pred_test_datasets[k][:,0]
        y = pred_test_datasets[k][:,1]
        sns.kdeplot(x=x, y=y, ax = axes[k,1], fill=True,cmap="Purples_r")
        axes[k,1].set_title(f"Trained model on test env {k}")

    plt.tight_layout()
    plt.show()

In [12]:
print(model.param)

ModelParameters
{'A': Array([[-1.03254139e-01, -1.75501421e-01, -1.26234695e-01,
        -6.07069917e-02, -2.06943631e-01,  1.00401655e-01,
        -7.20999613e-02, -1.45803079e-01,  2.01865047e-01,
         2.38311738e-01,  1.64960429e-01,  2.00756595e-01,
        -1.84132740e-01,  2.35010326e-01,  3.42388451e-02,
         2.05534846e-01,  9.91943106e-02, -2.37524912e-01,
         2.02464238e-01, -1.02153577e-01,  1.32919490e-01,
        -6.98035434e-02, -3.68954986e-02,  4.68418226e-02,
        -7.29106218e-02,  1.17525131e-01,  1.62746683e-01,
         1.17594779e-01,  2.55176723e-01,  1.72920763e-01,
        -2.28066638e-01, -1.27980724e-01, -7.80851692e-02,
        -1.01296715e-01, -8.01112652e-02, -1.17293306e-01,
         6.11776747e-02,  2.03506842e-01, -2.31550217e-01,
         1.27919093e-01, -1.22368075e-01, -1.55389935e-01,
        -1.02886856e-01,  2.58161962e-01, -1.43182531e-01,
        -1.00035429e-01,  2.01483220e-01,  1.62626550e-01,
        -1.62705705e-01, -1.821576

In [13]:
print(true_model.param)

ModelParameters
{'log_reversion': Array([-0., -0., -0., -0.,  0.,  0.,  0., -0., -0.,  0., -0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0., -0., -0.], dtype=float32),
 'mlp_0': Array([[1., 0.],
       [0., 1.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.],
       [0., 0.]], dtype=float32),
 'mlp_1': Array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.]], dtype=float32),
 'mlp_b_0': Array([ 0., -0.], dtype=float32)}
